### <b>Table of Content</b>

0. Import functions

1. Load data





5. Save models for later use

6. Conclusion

In [ ]:
URL = "https://docs.google.com/spreadsheets/d/1MiunF_O8eNWIcfaOA4PVm668RN7FgLNA0a6U4LWf5Bk/edit?usp=sharing"